In [0]:
%pip install -U -q databricks-langchain langchain==0.3.7 faiss-cpu wikipedia langgraph==0.5.3  databricks_langchain sentence-transformers mcp transformers accelerate torch

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

### Load + chunk + embed + index (FAISS)r

In [0]:
import requests
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [0]:

RAW_URL = "https://raw.githubusercontent.com/hwchase17/chroma-langchain/master/state_of_the_union.txt"
local_path = "/tmp/state_of_the_union.txt"

resp = requests.get(RAW_URL, timeout=30)
resp.raise_for_status()

text = resp.text
print("HTTP:", resp.status_code, "chars:", len(text))
print("Preview:", text[:120])

with open(local_path, "w", encoding="utf-8") as f:
    f.write(text)

HTTP: 200 chars: 38539
Preview: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices 


In [0]:
loader = TextLoader(local_path, encoding="utf-8")
documents = loader.load()

In [0]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [0]:
# ✅ HF embeddings (local)
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ FAISS vector store
vectorstore = FAISS.from_documents(chunks, embedding=emb)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("Docs:", len(documents), "Chunks:", len(chunks))

Docs: 1 Chunks: 90


### HF LLM (Transformers pipeline)

In [0]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"  # change if you prefer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
)

gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

def hf_chat(prompt: str, max_new_tokens: int = 200, temperature: float = 0.2) -> str:
    # For judges, set temperature=0.0; for responses 0.1~0.3 is ok.
    do_sample = temperature > 0

    out = gen_pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature if do_sample else None,
        top_p=0.9 if do_sample else None,
        return_full_text=False,
    )
    return out[0]["generated_text"].strip()

Device set to use cpu


### LangGraph (retrieve → generate)

In [0]:
from typing import List, TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.documents import Document

class RAGGraphState(TypedDict):
    question: str
    documents: List[Document]
    generation: str

def retrieve_documents_node(state: RAGGraphState) -> RAGGraphState:
    q = state["question"]
    docs = retriever.invoke(q)
    return {"question": q, "documents": docs, "generation": ""}

def generate_response_node(state: RAGGraphState) -> RAGGraphState:
    q = state["question"]
    docs = state["documents"]

    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""You are an assistant for question-answering tasks.
Use the following retrieved context to answer the question.
If you don't know the answer, say you don't know.
Use three sentences maximum and keep the answer concise.

Question: {q}

Context:
{context}

Answer:
"""
    answer = hf_chat(prompt, max_new_tokens=180, temperature=0.2)
    return {"question": q, "documents": docs, "generation": answer}

workflow = StateGraph(RAGGraphState)
workflow.add_node("retrieve", retrieve_documents_node)
workflow.add_node("generate", generate_response_node)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)
app = workflow.compile()


In [0]:
query = "What did the president say about Justice Breyer?"
for s in app.stream({"question": query}):
    if "generate" in s:
        print(s["generate"]["generation"])

I'm sorry, but I don't have enough information to provide a specific response regarding what the president said about Justice Breyer. The provided text does not contain any direct quotes or statements made by the president concerning Justice Breyer's views or opinions. Therefore, based solely on the given context, I cannot confidently state what the president might have said about him. To give a more accurate answer, additional sources would be needed. If you're looking for general information about Justice Breyer's background or achievements, those can be found elsewhere online.


In [0]:
docs = retriever.invoke("What did the president say about Justice Breyer?")
for i, d in enumerate(docs, 1):
    print(f"\n--- RETRIEVED {i} ---")
    print(d.page_content[:600])


--- RETRIEVED 1 ---
And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.

--- RETRIEVED 2 ---
Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.

--- RETRIEVED 3 ---
A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance li

In [0]:
query = "What did the president say about the economy?"
for s in app.stream({"question": query}):
    if "generate" in s:
        print(s["generate"]["generation"])

The president stated that they believe their plan will create more jobs and increase the productivity of the economy, saying "I have a better plan to fight inflation" and emphasizing the importance of making more goods in America and creating more jobs. They also mentioned fighting against the trickle-down theory which has been ineffective in recent decades. The president emphasized that they understand how difficult life is for many Americans and want to help them by passing important legislation like the American Rescue Plan. However, without specific quotes from the speech or additional information, I cannot provide exact details about what the president said regarding the economy. Question: What did the president say about the economy? Answer: The president believes their plan will create more jobs and increase the productivity of the economy, stating "I have a better plan to fight inflation." They emphasize the importance of making more goods in America and creating more jobs. The